## Global Model Based Eccentricity Method 1

$
\begin{equation}\label{eq:Ecc2}
\xi_i = \frac{d(\mathcal{H}^i_{{\mathcal M}}, \mathcal{H}_{\mu_i})}{\sum _{j=1}^{k} d(\mathcal{H}^j_{{\mathcal M}}, \mathcal{H}_{\mu_j})},
\end{equation}
$

Where $\mathcal{H}_{\mu_i}$ and $\mathcal{H}_{\mu_j}$ are respectively, the Hessian matrices of local models $\mu_i$ and $\mu_j$, for $j=1, 2,\ldots, k$, and $\mathcal{H}^i_{{\mathcal M}}$ and $\mathcal{H}^j_{{\mathcal M}}$ are the Hessian matrices of global model $\mathcal M$ calculated respectively on the validation sets of clients $i$ and $j$, for $j=1, 2,\ldots, k$,  and $d(.,.)$ is the Euclidean distance between two Hessian matrices. 


In [ ]:
glo_matrix_dict = {
        key: {"iso" : torch.load("hessians/iso/" + str(key) + ".pth"),
             "fed" : torch.load("hessians/fed/" + str(key) + ".pth")}
        for key in client_ids
    }

In [ ]:
distances = []
for client in client_ids:
    distance = accumulated_proximity(glo_matrix_dict[client]["iso"],glo_matrix_dict[client]["fed"], euclidean_distance)
    distances.append(distance)
    #print(client,acc_proximity, 2*acc_proximity/full_acc_proximity_p)

In [ ]:
full_acc = sum(distances)

In [ ]:
ecc_dis = []
for i in distances:
    ecc_dis.append(round((i/full_acc).item(),4))
ecc_diss = {"client id": client_ids, "hess_dis": ecc_dis}
ecc_diss = pd.DataFrame(ecc_diss)
ecc_diss.to_csv("insights/eccentricity_global_model_based_1.csv" , index=False)

## Global Model Based Eccentricity Method 2

$
\begin{equation}\label{eq:Ecc3}
\xi_i = \frac{2\sum_{j=1}^{k}d((\mathcal{H}^i_{\mathcal {M}} - \mathcal{H}_{\mu_i}), (\mathcal{H}^j_{\mathcal {M}} - \mathcal{H}_{\mu_j}))}{\sum_{l=1}^{k}\sum _{j=1}^{k} d((\mathcal{H}^l_{{\mathcal M}} - \mathcal{H}_{\mu_l}), (\mathcal{H}^j_{{\mathcal M}} - \mathcal{H}_{\mu_j}))},
\end{equation}
$

Where $\mathcal{H}_{\mu_i}$, $\mathcal{H}_{\mu_l}$ and $\mathcal{H}_{\mu_j}$ are respectively, the Hessian matrices of local models $\mu_i$, $\mu_l$ and $\mu_j$, for $l, j=1, 2,\ldots, k$, and $\mathcal{H}^i_{{\mathcal M}}$, $\mathcal{H}^l_{{\mathcal M}}$ and $\mathcal{H}^j_{{\mathcal M}}$ are the Hessian matrices of global model $\mathcal M$ calculated respectively on the validation sets of clients $i$, $l$ and $j$, for $l, j=1, 2,\ldots, k$,  and $d(.,.)$ is the Euclidean distance between the two matrices obtained by subtracting the corresponding Hessian matrices. 


In [ ]:
full_matrix_dict = {
        key: {"iso" : torch.load("hessians/iso/" + str(key) + ".pth"),
             "fed" : torch.load("hessians/fed/" + str(key) + ".pth")}
        for key in client_ids
    }
diff_matrix_dict = {key : full_matrix_dict[key]["fed"] - full_matrix_dict[key]["iso"]
                        for key in full_matrix_dict}

In [ ]:
proximities = []
for i in client_ids:
    distances = []
    for j in client_ids:
        distance = accumulated_proximity(diff_matrix_dict[i],diff_matrix_dict[j],manhattan_distance)
        distances.append(distance)
    print(round(sum(distances).item(),4))
    proximities.append(sum(distances).item())

In [ ]:
full_proximity = sum(proximities)

In [ ]:
full_proximity = sum(proximities)

ecc = []
for i in proximities:
    ecc.append(round((2 * i/full_proximity),4))
    
eccentricities_3 = {"client id": client_ids, "eccentricity": ecc}
eccentricities_3 = pd.DataFrame(eccentricities_3)
eccentricities_3.to_csv("insights/eccentricities_global_model_based_2.csv" , index=False)

## Eccentricity Frobenius Method 3

In [ ]:
matrix_dict = {
        key: torch.load("hessians/iso/" + str(key) + ".pth")
        for key in client_ids
    }

In [ ]:
eccentricities = []

for client in client_ids:
    client_matrix = torch.load("hessians/iso/" + str(client) + ".pth")
    acc_norm = 0.0
    for key in matrix_dict:
        norm = torch.norm(client_matrix - matrix_dict[key] , 'fro')
        acc_norm += norm
    
    eccentricities.append(round(acc_norm.item(),4))
    print(round(acc_norm.item(),4))

In [ ]:
full_ecc = []
full_acc_proximity = sum(eccentricities)*2

for item in eccentricities:
    full_ecc.append(2*(item/full_acc_proximity))

In [ ]:
ecc_hessian = {"client id": client_ids, "hess_ecc": full_ecc}
ecc_hessian = pd.DataFrame(ecc_hessian)
ecc_hessian.to_csv("insights/eccentricity_with_forb_norm.csv" , index=False)